<a href="https://colab.research.google.com/github/MatheusSilvaPains/Gerador-de-Arquivo-CSV/blob/main/gerar_zip_dos_arquivos_CSV_para_enturma%C3%A7%C3%A3o_em_lote_(por_bimestre).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Célula 1: Importação das Bibliotecas

# Importação das bibliotecas
import csv  # Para manipulação de arquivos CSV
import pandas as pd  # Para manipulação de dados em formato tabular
import zipfile  # Para criação de arquivos ZIP
import os  # Para interações com o sistema operacional
from google.colab import files  # Para upload e download de arquivos no Google Colab
from openpyxl import load_workbook
import zipfile

In [ ]:
# @title Célula 2: Função de Upload

# # Upload do arquivo Excel
# uploaded_files = files.upload()
# excel_filename = next(iter(uploaded_files))  # Extrai o nome do arquivo do dicionário

# Caminho para o arquivo Excel
excel_filename = "/content/Enturmação_Programacao_Academica_2025_Entrada1.xlsx"

In [ ]:
# @title Célula 3: Extrair as colunas "Turmas" de cada aba

# Abra o arquivo Excel
xls = pd.ExcelFile(excel_filename)

# Crie um dicionário para armazenar as planilhas modificadas
planilhas_modificadas = {}

# Percorra todas as abas
for aba in xls.sheet_names:
    # Carregar a aba como DataFrame
    df = pd.read_excel(excel_filename, sheet_name=aba)

    # Filtrar todas as colunas que contêm 'Turmas' no nome
    colunas_turmas = [col for col in df.columns if 'Turmas' in col]

    # Se encontrar colunas com 'Turmas', selecione apenas essas
    if colunas_turmas:
        df = df[colunas_turmas]

        # Armazenar a aba modificada no dicionário
        planilhas_modificadas[aba] = df
    else:
        print(f'Aba {aba} não possui nenhuma coluna com "Turmas".')

# Agora, vamos criar uma nova lógica para dividir cada coluna de cada aba em novas abas

# Salvar as planilhas modificadas em um novo arquivo Excel
arquivo_modificado = 'arquivo_modificado.xlsx'
with pd.ExcelWriter(arquivo_modificado, engine='openpyxl') as writer:
    # Para cada aba com as colunas "Turmas" filtradas
    for aba, df in planilhas_modificadas.items():
        # Iterar sobre cada coluna e criar uma nova aba
        for i, coluna in enumerate(df.columns, start=1):
            # Criar um DataFrame para a coluna
            nova_aba_df = df[[coluna]]

            # Criar um nome para a nova aba, por exemplo: "ADS1", "ADS2", etc.
            novo_nome_aba = f"{aba}{i}"

            # Escrever o DataFrame da nova aba no arquivo Excel
            nova_aba_df.to_excel(writer, sheet_name=novo_nome_aba, index=False)

    # Obter o workbook do arquivo para manipulação do tamanho das células
    workbook = writer.book

    # Ajustar a largura das colunas para o tamanho do conteúdo
    for aba in workbook.sheetnames:
        worksheet = workbook[aba]

        for col in worksheet.columns:
            max_length = 0
            column = col[0].column_letter  # Obtém a letra da coluna

            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass

            adjusted_width = (max_length + 2)  # Ajusta a largura da coluna (adiciona um espaço extra)
            worksheet.column_dimensions[column].width = adjusted_width

# Salvar o arquivo final com as alterações
workbook.save(arquivo_modificado)

print(f"Arquivo modificado com sucesso. Salvo como {arquivo_modificado}.")
# files.download(arquivo_modificado)

Aba Relatório de Referância não possui nenhuma coluna com "Turmas".
Arquivo modificado com sucesso. Salvo como arquivo_modificado.xlsx.


In [ ]:
# @title Célula 4: Excluir 'Trimestre de Ingresso' e as 2 células abaixo em todas as colunas "Turmas"

# Função para substituir todas as colunas pela palavra 'Turma'
def substituir_nome_coluna_por_turma(sheet):
    for col in sheet.columns:
        sheet[col[0].column_letter + '1'] = 'TURMA'

# Função para excluir as células 'Trimestre de Ingresso' e as duas células abaixo
def excluir_trimestre_ingresso(sheet):
    for col in sheet.columns:
        for cell in col:
            if cell.value == 'Trimestre de Ingresso':
                sheet.delete_rows(cell.row, 3)  # Exclui a célula e as duas abaixo dela
                break  # Interrompe a busca após excluir as células

# Função para ajustar a largura das colunas
def ajustar_largura_colunas(sheet):
    for col in sheet.columns:
        max_length = 0
        column = col[0].column_letter  # Obtém a letra da coluna
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass

        adjusted_width = (max_length + 2)  # Ajusta a largura da coluna (adiciona um espaço extra)
        sheet.column_dimensions[column].width = adjusted_width

# Função para alterar "RA" para "ALUNO" em todas as abas
def alterar_ra_para_aluno(sheet):
    for col in sheet.columns:
        for cell in col:
            if cell.value == 'RA':  # Verifica se a célula contém 'RA'
                cell.value = 'ALUNO'  # Substitui 'RA' por 'ALUNO'

# Função principal para processar o arquivo
def processar_arquivo(arquivo):
    # Carregar o arquivo Excel modificado
    workbook = load_workbook(arquivo)

    # Percorrer todas as abas do workbook
    for aba in workbook.sheetnames:
        sheet = workbook[aba]

        # Chama a função para substituir o nome das colunas para 'Turma'
        substituir_nome_coluna_por_turma(sheet)

        # Chama a função para excluir 'Trimestre de Ingresso' e as duas células abaixo
        excluir_trimestre_ingresso(sheet)

        # Chama a função para ajustar a largura das colunas
        ajustar_largura_colunas(sheet)

        # Chama a função para alterar 'RA' para 'ALUNO'
        alterar_ra_para_aluno(sheet)

    # Salvar as alterações no arquivo Excel
    workbook.save(arquivo)
    print(f"As alterações foram feitas com sucesso em {arquivo}.")

# Chame a função principal passando o nome do arquivo modificado
processar_arquivo('arquivo_modificado.xlsx')

# files.download('arquivo_modificado.xlsx')


As alterações foram feitas com sucesso em arquivo_modificado.xlsx.


In [ ]:
# @title Célula 5: Função para renomear as cinco primeiras colunas para DISCIPLINA, TURMA, ANO, PERIODO, ALUNO

# Função para buscar a célula "ALUNO" e mover seus valores e os valores abaixo para a coluna E, removendo células em branco acima
def mover_coluna_aluno_para_e(sheet):
    # Percorrer todas as colunas e linhas para encontrar a célula com o valor "ALUNO"
    for col in sheet.iter_cols():
        for cell in col:
            if cell.value == "ALUNO":
                aluno_col = cell.column  # Coluna onde está a célula "ALUNO"
                aluno_row = cell.row     # Linha onde está a célula "ALUNO"

                # Criar uma lista para armazenar os valores da coluna "ALUNO" e abaixo
                valores = [sheet.cell(row=i, column=aluno_col).value for i in range(aluno_row, sheet.max_row + 1)]

                # Limpar os valores da coluna original
                for i in range(aluno_row, sheet.max_row + 1):
                    sheet.cell(row=i, column=aluno_col).value = None

                # Colocar os valores na coluna "E" sem espaços em branco acima
                for i, valor in enumerate(valores, start=1):  # Começa na linha 1 para garantir que suba para o topo da coluna E
                    sheet[f'E{i}'] = valor

                return  # Interrompe a função após encontrar e mover a coluna "ALUNO"
def inserir_coluna_a_esquerda(sheet):
    sheet.insert_cols(1)  # Insere uma nova coluna na posição 1

# Função para renomear as cinco primeiras colunas para DISCIPLINA, TURMA, ANO, PERIODO, ALUNO
def renomear_primeiras_colunas(sheet):
    # Lista com os novos nomes para as primeiras cinco colunas
    novos_nomes = ["DISCIPLINA", "TURMA", "ANO", "PERIODO", "ALUNO"]

    # Renomear cada uma das cinco primeiras colunas
    for i, nome in enumerate(novos_nomes):
        coluna = sheet.cell(row=1, column=i + 1)  # Linha 1 e coluna i+1
        coluna.value = nome

# Função principal para processar o arquivo e aplicar todas as modificações
def processar_arquivo_com_modificacoes(arquivo):
    # Carregar o arquivo Excel
    workbook = load_workbook(arquivo)

    # Percorrer todas as abas do workbook
    for aba in workbook.sheetnames:
        sheet = workbook[aba]

        # Inserir coluna à esquerda em cada aba
        inserir_coluna_a_esquerda(sheet)

        # Chamar outras funções para modificações
        substituir_nome_coluna_por_turma(sheet)
        excluir_trimestre_ingresso(sheet)
        ajustar_largura_colunas(sheet)
        alterar_ra_para_aluno(sheet)

        # Mover a coluna "ALUNO" e as células abaixo para a coluna E, removendo células em branco acima
        mover_coluna_aluno_para_e(sheet)

        # Renomear as cinco primeiras colunas
        renomear_primeiras_colunas(sheet)

    # Salvar as alterações no arquivo Excel
    workbook.save(arquivo)
    print(f"Todas as alterações foram feitas com sucesso em {arquivo}.")

# Chame a função principal passando o nome do arquivo modificado
processar_arquivo_com_modificacoes('arquivo_modificado.xlsx')

# files.download('arquivo_modificado.xlsx')

Todas as alterações foram feitas com sucesso em arquivo_modificado.xlsx.


In [ ]:
# @title Célula 6: Função para preencher as colunas DISCIPLINA, ANO e PERIODO com valores da coluna TURMA

# Função para preencher as colunas DISCIPLINA, ANO e PERIODO com valores da coluna TURMA
def preencher_dados_a_partir_de_turma(arquivo):
    # Carregar o arquivo Excel
    workbook = load_workbook(arquivo)

    for aba in workbook.sheetnames:
        sheet = workbook[aba]

        # Identificar as colunas DISCIPLINA, TURMA, ANO e PERIODO
        disciplina_coluna = None
        turma_coluna = None
        ano_coluna = None
        periodo_coluna = None

        for col in sheet.iter_cols(1, sheet.max_column):
            header = col[0].value
            if header == "DISCIPLINA":
                disciplina_coluna = col[0].column
            elif header == "TURMA":
                turma_coluna = col[0].column
            elif header == "ANO":
                ano_coluna = col[0].column
            elif header == "PERIODO":
                periodo_coluna = col[0].column

        # Preencher DISCIPLINA, ANO e PERIODO com valores extraídos de TURMA
        if disciplina_coluna and turma_coluna and ano_coluna and periodo_coluna:
            for row in range(2, sheet.max_row + 1):
                turma_valor = sheet.cell(row=row, column=turma_coluna).value
                if turma_valor and len(str(turma_valor)) >= 17:
                    # Preenche DISCIPLINA com os 11 primeiros dígitos de TURMA
                    sheet.cell(row=row, column=disciplina_coluna).value = str(turma_valor)[:11]
                    # Preenche ANO com os dígitos nas posições 13 a 16 de TURMA
                    sheet.cell(row=row, column=ano_coluna).value = str(turma_valor)[12:16]
                    # Preenche PERIODO com o caractere na posição 17 de TURMA
                    sheet.cell(row=row, column=periodo_coluna).value = str(turma_valor)[16]

        # Chama a função para ajustar a largura das colunas
        ajustar_largura_colunas(sheet)

    # Salvar o arquivo com as alterações
    workbook.save(arquivo)
    print(f"As colunas DISCIPLINA, ANO e PERIODO foram preenchidas com valores de TURMA em {arquivo}.")

# Chame a função passando o nome do arquivo
preencher_dados_a_partir_de_turma('arquivo_modificado.xlsx')
# files.download('arquivo_modificado.xlsx')


As colunas DISCIPLINA, ANO e PERIODO foram preenchidas com valores de TURMA em arquivo_modificado.xlsx.


In [ ]:
# @title Célula 7: Função para gerar CSV

def gerar_csv_e_zip_por_turma(arquivo_excel, pasta_saida="csv_por_turma", nome_zip="csv_por_turma.zip"):
    # Garantir que a pasta de saída exista
    os.makedirs(pasta_saida, exist_ok=True)

    # Abrir o arquivo Excel
    workbook = load_workbook(arquivo_excel)

    for aba in workbook.sheetnames:
        # print(f"Processando aba: {aba}")
        sheet = workbook[aba]

        # Identificar colunas
        turma_coluna = None
        aluno_coluna = 5  # Coluna "E" é fixa
        for col in sheet.iter_cols(1, sheet.max_column):
            if col[0].value == "TURMA":
                turma_coluna = col[0].column
                break

        if not turma_coluna:
            print(f"Aba {aba} não possui coluna TURMA. Ignorando...")
            continue

        # Criar um DataFrame para alunos
        alunos = [sheet.cell(row=row, column=aluno_coluna).value for row in range(2, sheet.max_row + 1)]
        alunos = [aluno for aluno in alunos if aluno is not None]

        # Identificar TURMAS únicas e validar os valores
        turmas_validas = {}
        turmas_invalidas = []  # Lista para armazenar valores inválidos
        for row in range(2, sheet.max_row + 1):
            turma_valor = sheet.cell(row=row, column=turma_coluna).value
            if turma_valor and len(str(turma_valor)) >= 17:
                turmas_validas[turma_valor] = {
                    "DISCIPLINA": turma_valor[:11],
                    "TURMA": turma_valor,
                    "ANO": turma_valor[12:16],
                    "PERIODO": turma_valor[16],
                }
            else:
                turmas_invalidas.append({"Linha": row, "TURMA": turma_valor})

        # Gerar CSVs para cada TURMA válida
        for turma, info in turmas_validas.items():
            dados_csv = []
            for aluno in alunos:
                dados_csv.append({
                    "DISCIPLINA": info["DISCIPLINA"],
                    "TURMA": info["TURMA"],
                    "ANO": info["ANO"],
                    "PERIODO": info["PERIODO"],
                    "ALUNO": aluno,
                })

            # Criar DataFrame e salvar como CSV
            df = pd.DataFrame(dados_csv)
            nome_arquivo = os.path.join(pasta_saida, f"{aba}_{turma}.csv")
            df.to_csv(nome_arquivo, sep=";", index=False)
            # print(f"CSV criado: {nome_arquivo}")

        # Gerar CSV para valores inválidos
        if turmas_invalidas:
            df_invalidas = pd.DataFrame(turmas_invalidas)
            nome_arquivo_invalidos = os.path.join(pasta_saida, f"{aba}_faltam_informacoes.csv")
            df_invalidas.to_csv(nome_arquivo_invalidos, sep=";", index=False)
            # print(f"CSV criado com informações faltando: {nome_arquivo_invalidos}")

    # Compactar todos os arquivos CSV em um único arquivo ZIP
    caminho_zip = os.path.join(pasta_saida, nome_zip)
    with zipfile.ZipFile(caminho_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(pasta_saida):
            for file in files:
                if file.endswith(".csv"):  # Apenas arquivos CSV
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, arcname=file)  # Adicionar ao ZIP
    print(f"Arquivo ZIP criado: {caminho_zip}")

    return caminho_zip

# Executar a função
zip_gerado = gerar_csv_e_zip_por_turma("arquivo_modificado.xlsx")

Arquivo ZIP criado: csv_por_turma/csv_por_turma.zip
